## Estruturas Criptográficas 2022/23
## TP2. Problema 2
### Grupo 7. Leonardo Berteotti e Paulo R. Pereira

TBC

In [ ]:
import hashlib, os 
from pickle import dumps
from sage.all import *

As classes **Ed** e **ed** foram dadas pelo docente.

In [100]:
class Ed(object):
    def __init__(self,p, a, d , ed = None):
        assert a != d and is_prime(p) and p > 3
        K         = GF(p) 
  
        A =  2*(a + d)/(a - d)
        B =  4/(a - d)
    
        alfa = A/(3*B) ; s = B

        a4 =  s^(-2) - 3*alfa^2
        a6 =  -alfa^3 - a4*alfa
        
        self.K = K
        self.constants = {'a': a , 'd': d , 'A':A , 'B':B , 'alfa':alfa , 's':s , 'a4':a4 , 'a6':a6 }
        self.EC = EllipticCurve(K,[a4,a6]) 
        
        if ed != None:
            self.L = ed['L']
            self.P = self.ed2ec(ed['Px'],ed['Py'])  # gerador do gru
        else:
            self.gen()
    
    def order(self):
        # A ordem prima "n" do maior subgrupo da curva, e o respetivo cofator "h" 
        oo = self.EC.order()
        n,_ = list(factor(oo))[-1]
        return (n,oo//n)
    
    def gen(self):
        L, h = self.order()       
        P = O = self.EC(0)
        while L*P == O:
            P = self.EC.random_element()
        self.P = h*P ; self.L = L

  
    
    def is_edwards(self, x, y):
        a = self.constants['a'] ; d = self.constants['d']
        x2 = x^2 ; y2 = y^2
        return a*x2 + y2 == 1 + d*x2*y2

    def ed2ec(self,x,y):      ## mapeia Ed --> EC
        if (x,y) == (0,1):
            return self.EC(0)
        z = (1+y)/(1-y) ; w = z/x
        alfa = self.constants['alfa']; s = self.constants['s']
        return self.EC(z/s + alfa , w/s)
    
    def ec2ed(self,P):        ## mapeia EC --> Ed
        if P == self.EC(0):
            return (0,1)
        x,y = P.xy()
        alfa = self.constants['alfa']; s = self.constants['s']
        u = s*(x - alfa) ; v = s*y
        return (u/v , (u-1)/(u+1))

In [101]:
class ed(object):
    def __init__(self,pt=None,curve=None,x=None,y=None):
        if pt != None:
            self.curve = pt.curve
            self.x = pt.x ; self.y = pt.y ; self.w = pt.w
        else:
            assert isinstance(curve,Ed) and curve.is_edwards(x,y)
            self.curve = curve
            self.x = x ; self.y = y ; self.w = x*y
    
    def eq(self,other):
        return self.x == other.x and self.y == other.y
    
    def copy(self):
        return ed(curve=self.curve, x=self.x, y=self.y)
    
    def zero(self):
        return ed(curve=self.curve,x=0,y=1)
    
    def sim(self):
        return ed(curve=self.curve, x= -self.x, y= self.y)
    
    def soma(self, other):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*self.w*other.w
        self.x, self.y  = (self.x*other.y + self.y*other.x)/(1+delta), (self.y*other.y - a*self.x*other.x)/(1-delta)
        self.w = self.x*self.y
        
    def duplica(self):
        a = self.curve.constants['a']; d = self.curve.constants['d']
        delta = d*(self.w)^2
        self.x, self.y = (2*self.w)/(1+delta) , (self.y^2 - a*self.x^2)/(1 - delta)
        self.w = self.x*self.y
        
    def mult(self, n):
        m = Mod(n,self.curve.L).lift().digits(2)   ## obter a representação binária do argumento "n"
        Q = self.copy() ; A = self.zero()
        for b in m:
            if b == 1:
                A.soma(Q)
            Q.duplica()
        return A

In [102]:
class EdDSA:
    storage = []
    
    def __init__(self, ed='ed448'):
        if(ed=='ed25519'):
            print('Algoritmo ed25519 definido.')
            self.setup_ed25519()
        else:
            print('Algoritmo ed448 definido.')
            self.setup_ed448()

    def setup_ed25519(self):
        p = 2^255-19
        K = GF(p)
        a = K(-1)
        d = -K(121665)/K(121666)
        #

        ed25519 = {
        'b'  : 256,
        'Px' : K(15112221349535400772501151409588531511454012693041857206046113283949847762202),
        'Py' : K(46316835694926478169428394003475163141307993866256225615783033603165251855960),
        'L'  : ZZ(2^252 + 27742317777372353535851937790883648493), ## ordem do subgrupo primo
        'n'  : 254,
        'h'  : 8
        }

        Px = ed25519['Px']; Py = ed25519['Py']

        E = Ed(p,a,d,ed=ed25519)
        G = ed(curve=E,x=Px,y=Py)
        l = E.order()[0]

        self.b = ed25519['b']
        self.requested_security_strength = 128
        self.E = E
        self.G = G
        self.l = l
        self.algorithm = 'ed25519'


    def setup_ed448(self):
        p = 2^448 - 2^224 - 1
        K = GF(p)
        a = K(1)
        d = K(-39081)

        ed448= {
        'b'  : 456,     ## tamanho das assinaturas e das chaves públicas
        'Px' : K(224580040295924300187604334099896036246789641632564134246125461686950415467406032909029192869357953282578032075146446173674602635247710) ,
        'Py' : K(298819210078481492676017930443930673437544040154080242095928241372331506189835876003536878655418784733982303233503462500531545062832660) ,                                          
        'L'  : ZZ(2^446 - 13818066809895115352007386748515426880336692474882178609894547503885) ,
        'n'  : 447,     ## tamanho dos segredos: os dois primeiros bits são 0 e o último é 1.
        'h'  : 4        ## cofactor
        }

        Px = ed448['Px']; Py = ed448['Py']

        E = Ed(p,a,d, ed=ed448)
        G = ed(curve=E,x=Px,y=Py)
        l = E.order()[0]

        self.b = ed448['b']
        self.requested_security_strength = 224
        self.E = E
        self.G = G
        self.l = l
        self.algorithm = 'ed448'

    #Função de hash a ser utilizada nas curvas de ED2556 e ED448
    def hash(self,data):
        if self.algorithm == 'ed25519':
            return hashlib.sha512(data).digest()
        else:
            return hashlib.shake_256(data).digest(912/8)

    #Função que determina a hash da chave privada
    def digest(self,d):
        h = self.hash(d)
        buffer = bytearray(h)
        return buffer

    def s_value(self,h):
        if self.algorithm == 'ed25519':
            return self.s_value_ed25519(h)
        else:
            return self.s_value_ed448(h)

    def s_value_ed25519(self,h):
        #Passar o valor recebido (octet string) para inteiro little endian
        digest = int.from_bytes(h, 'little')
        bits = [int(digit) for digit in list(ZZ(digest).binary())]
        x = 512 - len(bits)
        while x != 0:
            bits = [0] + bits
            x = x-1
        #Manipulação dos bits do inteiro segundo o algoritmo
        bits[0] = bits[1] = bits[2] = 0
        bits[self.b-2] = 1
        bits[self.b-1] = 0
        #Junção de todos os bits
        bits = "".join(map(str, bits))
        #valor em inteiro em formato little endian
        s = int(bits[::-1], 2)
        return s

    def s_value_ed448(self,h):
        #Passar o valor recebido (octet string) para inteiro little endian
        digest = int.from_bytes(h, 'little')
        bits = [int(digit) for digit in list(ZZ(digest).binary())]
        x = 512 - len(bits)
        while x != 0:
            bits = [0] + bits
            x = x-1
        #Manipulação dos bits do inteiro segundo o algoritmo
        bits[0] = bits[1] = 0
        bits[self.b-9] = 1
        for i in bits[self.b-8:self.b]:
            bits[i] = 0
        #Junção de todos os bits
        bits = "".join(map(str, bits))
        #valor em inteiro em formato little endian
        s = int(bits[::-1], 2)
        return s

    #Função de encoding - compressão de ponto
    def encoding(self,Q, n):
        x, y = Q.x, Q.y
        #armazena ponto completo no indice que recebeu
        self.storage.insert(n,(x,y))
        return x
    
    #Função de decoding - descompressão de ponto
    def decoding(self,n):
        #devolve o ponto que estava no índice que recebeu como parâmetro
        Q = self.storage[n]
        return Q
    

    #Função de geração do par de chaves
    def generate_keys(self):
        #Gerar a private key
        d = os.urandom(self.b//8)
        #Geração do valor s
        digest = self.digest(d) 
        if self.algorithm == 'ed25519':
            bytes_length = 32
        else:
            bytes_length = 57

        hdigest1 = digest[:bytes_length]
        s = self.s_value(hdigest1)
        #Gerar a chave pública
        T = self.G.mult(s)

        #Compressão da chave publica - passagem para octet string
        Q = self.encoding(T,0)
        Q = int(Q).to_bytes(bytes_length, 'little')
        return d, Q


    def dom4(self, f, context): 
        init_string = []
        context_octets = []
        
        for c in context:
            context_octets.append(format(ord(c), "08b"))
        context_octets = ''.join(context_octets)

        for c in "SigEd448":
            init_string.append(format(ord(c), "08b"))
        init_string = ''.join(init_string)

        bits_int = int(init_string + format(f, "08b") + format(len(context_octets), "08b") + context_octets, 2)
        byte_array = bits_int.to_bytes((bits_int.bit_length() + 7) // 8, 'little')
        
        return byte_array
        
    #Função de assinatura de uma mensagem
    def signature(self,M,d,Q,context = ''):
        #Determina hash da chave privada
        digest = self.digest(d)

        if self.algorithm == 'ed25519':
            bytes_length = 32
            hashPK = digest[bytes_length:]
            hashPK_old = digest[:bytes_length]
            r = self.hash(hashPK+M)
        else:
            bytes_length = 57
            hashPK = digest[bytes_length:]
            hashPK_old = digest[:bytes_length]
            r = self.hash(self.dom4(0, context)+hashPK+M)
        #Determinar valor r
        r = int.from_bytes(r, 'little')
        #Determinar ponto R,comprimi-lo e transforma em octet string
        R = self.G.mult(r)
        Rx = self.encoding(R,1)
        R = int(Rx).to_bytes(bytes_length, 'little')
        #Determinar valor s
        s = self.s_value(hashPK_old)
        
        if self.algorithm == 'ed25519':
            #Determinar hash da octet string R||Q||M
            hashString = self.hash(R+Q+M)
        else:
            #Determinar hash da octet string dom4(0,context)||R||Q||M
            hashString = self.hash(self.dom4(0, context)+R+Q+M)

        hashString = int.from_bytes(hashString, 'little')
        # Expressão para determinar S - S = (r + SHA-512(R || Q || M) * s) mod n
        S = mod(r + hashString * s,self.l)
        #Valor de S em octet string
        S = int(S).to_bytes(bytes_length, 'little')
        #Assinatura final
        signature = R+S
        return signature
    
    #Função de verificação de uma mensagem
    def verify(self,M,A,Q, context = ''):
        if self.algorithm == 'ed25519':
            bytes_length = 32
        else:
            bytes_length = 57
        #Retira valores R e S da assinatura A
        R = A[:bytes_length]
        S = A[bytes_length:]
        s = int.from_bytes(S, 'little')
        #Verificação dos processos de decoding das variaveis S, R e Q
        if (s >= 0 and s < self.l):
            (Rx, Ry) = self.decoding(1)
            (Qx, Qy) = self.decoding(0)
            if(Rx != None and Qx != None):
                res = True
            else: return False
        else: return False

        #Determinar valor t
        if self.algorithm == 'ed25519':
            digest = self.hash(R+Q+M)
        else:
            digest = self.hash(self.dom4(0, context)+R+Q+M)
            
        t = int.from_bytes(digest, 'little')
        #Determinar variaveis a serem usadas na expressão de verificação
        value = 2^3
        R = int.from_bytes(R, 'little')
        Q = int.from_bytes(Q, 'little')
        R = ed(curve=self.E,x=Rx,y=Ry)
        Q = ed(curve=self.E,x=Qx,y=Qy)
        #Determinar valores da condição de verificação - [2^c * S]G == [2^c]R + (2^c * t)Q
        part1 = self.G.mult(value*s)
        part2 = R.mult(value)
        part3 = Q.mult(value*t)
        part2.soma(part3)
        #Verificação da condição de verificação
        if part1.eq(part2):
            res = True
        else :
            res = False
        return res

In [103]:
edDSA = EdDSA()
signed_message = "Esta mensagem esta assinada"
unsigned_message = "Esta mensagem nao esta assinada"
print("Mesangem para ser assianda: " + signed_message)
privateKey, publicKey = edDSA.generate_keys()
print("SK: ")
print(privateKey)
print("PK: ")
print(publicKey)
print()
assinatura = edDSA.signature(dumps(signed_message), privateKey, publicKey, 'contexto')
print("Assinatura: ")
print(assinatura)
print()
print("Verificação da autenticação da mensagem assinada")
if edDSA.verify(dumps(signed_message), assinatura, publicKey, 'contexto')==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada!")
    
print()
print("Verificação da autenticação da mensagem não assinada")
if edDSA.verify(dumps(unsigned_message), assinatura, publicKey, 'contexto')==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada!")

Algoritmo ed448 definido.
Mesangem para ser assianda: Esta mensagem esta assinada
SK: 
b'wq\x1f=\xf7\xf08$7qL\xe1\xfaD*\xc1\x9e%]\x1b\x8f\xc9\x90v\xf5\xeb*\xd1\x1bD\xb7\n\x8cV\x00_\x93\xce\x88\xa1_K$\xaf\x8a\xe21\x98\xef\xb0\x97\x8bD\x86\xedR\xe8'
PK: 
b'\x10\x1aF"\xad-\xc5.\xe1W\xdd\xf6\xfb\xc2\xdc\xf9}\x00\xd7\\\xa7\xf0M\xb5\xca_\xc8\xe8\x88\x0fT8\x10\x18\xc6\xba\x8d\xb2\xfbr\xb4\xbaV0\xc6\xa5S\xb8\xfe\x92l)_9C-\x00'

Assinatura: 
b'\x9a\x02\xe7\xf4\xc6\xb1\x05\r\xd9\xc8Dy\xc8\xe5\xe3\xb5z\xfe9\x02:\x16\x11,\x96\xa3\xfc&\xd9\xabe\xba\xd8\xa9\xb6\xca\xa1\xc3\x98\x87er\x8b_\xa9\xbd`\xe7\x93\xf7\x18\xd1\xf0\x8f\x88\xb4\x00+x\x85\xdd\x0cE\xf4\xcc\xc8\xdd\x81\x96\xadas\xdb\xa9a\x93\xa0\xf5\x83(\x15\xe3\x81\x05\xaeE\x01\x13\xfaZ\x8d\xd6t\x0e\xc0\x17e\xee\x18V\xc9:`\xadX/\xb6\x8c\xb1\x12\\\x89 \x00'

Verificação da autenticação da mensagem assinada
Mensagem autenticada!

Verificação da autenticação da mensagem não assinada
Mensagem não autenticada!


In [104]:
edDSA = EdDSA('ed25519')
signed_message = "Esta mensagem esta assinada"
unsigned_message = "Esta mensagem nao esta assinada"
print("Mesangem para ser assianda: " + signed_message)
privateKey, publicKey = edDSA.generate_keys()
print("SK: ")
print(privateKey)
print("PK: ")
print(publicKey)
print()
assinatura = edDSA.signature(dumps(signed_message), privateKey, publicKey, 'contexto')
print("Assinatura: ")
print(assinatura)
print()
print("Verificação da autenticação da mensagem assinada")
if edDSA.verify(dumps(signed_message), assinatura, publicKey, 'contexto')==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada!")
    
print()
print("Verificação da autenticação da mensagem não assinada")
if edDSA.verify(dumps(unsigned_message), assinatura, publicKey, 'contexto')==True:
    print("Mensagem autenticada!")
else:
    print("Mensagem não autenticada!")

Algoritmo ed25519 definido.
Mesangem para ser assianda: Esta mensagem esta assinada
SK: 
b'\xc9\x9f\x19\xf9z\xa2\xe31\xbcK\xd2\x0b$*\xceD!x\x867\xdb+C~(\xff\x1d\xa7\x96\x07*q'
PK: 
b'%\xe0\xe0\xa1\xea0\x13\x94\xf5\xf1\x01\x9e\x12\xe5\x04\xc1K\x03\x1a\xc7\xc6)\xe5\x00\xb0b\xec\x19W\xb6\\<'

Assinatura: 
b'\xea\xab\xc9\xc1\xd3p\t\x99\xc2\x9b1\x82\xa1<\xc1K\x94m\xf6\xc4\xa9S\x81\x8e\xd6\xc0 o\xaf\x1f\xe4_9\xae\x94&y6\x05\xe5\xef\x085\xae{P\x8f\x85,\xfa&\x8csy 5\x88\xc5d\xec\xceN\xb0\x06'

Verificação da autenticação da mensagem assinada
Mensagem autenticada!

Verificação da autenticação da mensagem não assinada
Mensagem não autenticada!
